In [0]:
dbutils.widgets.text(name="env", defaultValue='', label='Enter the environment in lower case')
env = dbutils.widgets.get("env")

In [0]:
checkpoint = spark.sql(f"DESCRIBE EXTERNAL LOCATION `checkpoints_{env}`").select("url").collect()[0][0]
landing = spark.sql(f"DESCRIBE EXTERNAL LOCATION `landing_{env}`").select("url").collect()[0][0]
bronze = spark.sql(f"DESCRIBE EXTERNAL LOCATION `bronze_{env}`").select("url").collect()[0][0]
silver = spark.sql(f"DESCRIBE EXTERNAL LOCATION `silver_{env}`").select("url").collect()[0][0]
gold = spark.sql(f"DESCRIBE EXTERNAL LOCATION `gold_{env}`").select("url").collect()[0][0]

In [0]:
print(f"checkpoint: {checkpoint}")
print(f"landing: {landing}")
print(f"bronze: {bronze}")
print(f"silver: {silver}")
print(f"gold: {gold}")

In [0]:
# 01 -- Removing duplicates
def remove_Dups(df):
    print('Removing Duplicate values: ', end='')
    df_dup = df.dropDuplicates()
    print('Success!')
    return df_dup

In [0]:
# 02 -- Handling NULLs (simple generic approach)
def handle_NULLs(df, columns):
    from pyspark.sql.types import StringType, NumericType
    print('Replacing NULLs... ', end='')

    # Detect types
    string_cols = [c.name for c in df.schema.fields 
                   if isinstance(c.dataType, StringType) and c.name in columns]
    numeric_cols = [c.name for c in df.schema.fields 
                    if isinstance(c.dataType, NumericType) and c.name in columns]

    df_filled = df
    if string_cols:
        df_filled = df_filled.fillna('Unknown', subset=string_cols)
    if numeric_cols:
        df_filled = df_filled.fillna(0, subset=numeric_cols)

    print('Success!')
    return df_filled